In [1]:
import requests
import json
import pandas as pd
import folium

In [2]:
def get_json(sido_code):
    body = {
        'ins_lat': '37.56682',
        'ins_lng': '126.97865', 
        
        'p_sido_cd': '{}'.format(sido_code if sido_code > 9 else "0{}".format(sido_code)),
        'p_gugun_cd': '',
        'in_biz_cd': '',
        'set_date': '', 
        'searchType': 'C',
        'all_store': 1,
        'iend': '1000',
        
    }
    url = 'https://www.starbucks.co.kr/store/getStore.do'
    r = requests.post(url, data=body)
    
    return json.loads(r.text)

In [3]:
if __name__ =="__main__":
    df = None
    
    for area in range(1,18):
        jo = get_json(area)
        
        if df is None:
            df = pd.json_normalize(jo, 'list')
        else:
            area_df = pd.json_normalize(jo, 'list')
            df = df.append(other= area_df, ignore_index=True, sort=False)
    
    df= df.astype({'lat': 'float', 'lot': 'float', 's_code': 'int'})
    df= df[['s_code', 's_name', 'sido_name', 'lat', 'lot']]
    df = df.sort_values(by="s_code")

    df

In [ ]:
map_osm = folium.Map(location=[36,127], zoom_start=7)

for index, row in df.iterrows():
    location = (row['lat'], row['lot'])
    folium.Marker(location, popup= row['s_name']).add_to(map_osm)